<a href="https://colab.research.google.com/github/lewyingshi/module2_lectures/blob/master/1_6_introduction_to_piping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [26]:
%%R
library(dplyr)

# Introduction to piping

## Review -- Piping with `dplyr`

You might be familiar with piping in R using the `%>%` operator from `dplyr`

In [27]:
%%R # magic command in python
pi %>% 
sqrt %>% 
round(2) %>% 
as.character

R[write to console]: Error in round(., 2) : non-numeric argument to mathematical function
Calls: <Anonymous> ... withVisible -> eval -> eval -> _fseq -> freduce -> <Anonymous>




Error in round(., 2) : non-numeric argument to mathematical function
Calls: <Anonymous> ... withVisible -> eval -> eval -> _fseq -> freduce -> <Anonymous>


## Making `pipeable` functions using `composable`

To get functions to be pipeable in Python, we need to wrap them in `pipeable` from the `composable` module

In [28]:
!pip install composable # Install if missing or in colab

In [29]:
from composable import pipeable

## Making some pipeable functions

Before I can recreate the R example, I need to make some pipeable functions.

## Making a pipeable `sqrt`

To make an existing function pipeable, I need to wrap or *decorate* it with `pipeable`.

In [30]:
import math as m
sqrt = pipeable(m.sqrt)

In [32]:
sqrt(2)

1.4142135623730951

In [31]:
2 >> sqrt

1.4142135623730951

## Some common functions are not actually functions

Some Python "functions" are not actually functions, but type constructors.  Examples include `str`, `float`, `int`, `list`, etc.  This also includes the most powerful type constructor of them all, `type`

In [35]:
type(str)

type

In [36]:
wont_always_work = pipeable(float)
3.5 >> sqrt >> wont_always_work

1.8708286933869707

## Use a `lambda` to create a pipeable type conversion function

To be safe, we need to wrap type constructors in a lambda, THEN `pipeable`

In [33]:
toStr = pipeable(lambda n: str(n))

In [34]:
3.5 >> toStr

'3.5'

## Piping and multiple arguments

Piped in data is inserted **on the right**

In [37]:
test = pipeable(lambda a, b: f"first:{a} second:{b}")

In [38]:
test(1,2)

'first:1 second:2'

In [39]:
1 >> test(2) # R pipes into the first arg, python pipes into the last

'first:2 second:1'

In [40]:
2 >> test(1)

'first:1 second:2'

In [42]:
# cannot pipe multiple functions in an argument
2 >> 1 >> test

<function <lambda> at 0x7fb6499f79d8>

## Rearranging argument order

The default `round` function uses `round(number, digits)`

In [43]:
round(m.pi, 2)

3.14

For piping, it is more convenient to switch the order.

In [44]:
rnd = pipeable(lambda d, n: round(n, d))

In [45]:
m.pi >> rnd(2) 

3.14

## Recreating the R example

In [49]:
m.pi >> sqrt >> rnd(2) >> toStr

'1.77'

In [50]:
# must wrap in parenthesis
(m.pi >> 
sqrt >> 
rnd(2) >> 
toStr)

'1.77'

## Hint 1: Wrap multi-line piped expressions in parentheses

In [51]:
(m.pi >> 
 sqrt >> 
 rnd(2) >> 
 toStr)

'1.77'

## Hint 2: Put the pipes at the start of a line

In [52]:
(m.pi 
 >> sqrt 
 >> rnd(2) 
 >> toStr
)

'1.77'

## Pipeable functions return functions when partially complete

Note that `pipeable` functions are curried, meaning they return functions if not provided with enough arguments.

In [64]:
threeArgs= pipeable(lambda a, b, c: f"first:{a} second:{b} third:{c}")

In [65]:
threeArgs("Bob")

<function <lambda> at 0x7fb64993b1e0>

In [66]:
threeArgs("Bob", "Alice")

<function <lambda> at 0x7fb64993b1e0>

## We can save and call a partial functions 

In [67]:
bob = threeArgs("Bob")

In [68]:
bob(2,3)

'first:{a} second:{b} third:{c}'

In [58]:
bobAndAlice = bob("Alice")

In [59]:
bobAndAlice(3)

'first:Bob second:Alice third:3'

## Example

Suppose that I round to two decimal places A LOT.  In this case it might be nice to have a specialized function

In [60]:
rndToTwo = rnd(2)

In [61]:
m.pi >> rndToTwo

3.14

<font color="red"><h1>Exercise 3</h1></font>

Here is a problem that you solved in a previous activity:

    The function `random` from the `random` module can be used to generate numbers between 0 and 1 at random. We want to return numbers between $a$ and $b$ at random, which can be accomplished using the formula $V = (b - a)*random() + a$.

    Write a lambda function that takes `a` and `b` as arguments are returns a number between `a` and `b` at random.
   
Note that we can name the parts the process as follows:

> b takeAway a >> times a random number >> subtract a

Let's redo this problem, but this time with piping; where we will make a pipeable function to perform each task.

In [ ]:
(b - a)*random() + a

In [74]:
from random import random
from composable import pipeable
takeAway = pipeable(lambda a, b: b - a)
timesRandom = pipeable(lambda c: c * random())
add = pipeable(lambda a, b: a + b)

In [76]:
# top-down design
def uniform(a, b):
    return ( b
            >> takeAway(a)
            >> timesRandom
            >> add(a)
            )

In [78]:
## A test function you should be able to pass when complete
assert all(1 <= uniform(1,2) <= 2 for i in range(10))